In [1]:
%load_ext autoreload
%autoreload 2

import polars as pl
import pandas as pd
from datetime import datetime
from lightweight_charts import Chart, JupyterChart
from IPython.display import display
from time import sleep
import ta

from jaref_bot.analysis.utils import create_ohlcv_from_trades

from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [19]:
symbol = 'BTC'
timeframe = '1h'

trades_df  = pl.read_parquet(f'./data/{symbol}_agg_trades.parquet')
ohlcv_df = create_ohlcv_from_trades(trades_df, timeframe=timeframe, calculate_cvd=True)

In [20]:
ohlcv_df.tail(3)

time,open,high,low,close,volume,n_trades,cvd_open,cvd_high,cvd_low,cvd_close
datetime[ms],f64,f64,f64,f64,f64,u32,f64,f64,f64,f64
2025-06-01 01:00:00,104446.49,104505.58,103991.37,104126.07,460.48201,27757,-220.77576,-212.94211,-386.38563,-382.79052
2025-06-01 02:00:00,104126.08,104324.21,103924.52,104195.12,310.1251,24116,-382.79047,-364.74156,-415.09758,-374.58178
2025-06-01 03:00:00,104195.13,104235.77,103971.7,104120.72,128.57099,10584,-374.48376,-367.71599,-393.9841,-383.22219


In [ ]:
# df_5min.rename(columns={'bucket': 'time'}, inplace=True)
# df_5min['time'] = df_5min['time'].astype(str)
# df_5min.drop(['exchange', 'symbol'], axis=1, inplace=True)

pdf = ohlcv_df.to_pandas()
pdf['time'] = pdf['time'].astype(str)

candles_df  = pdf[['time', 'open', 'high', 'low', 'close', 'volume']]
cvd_df = pdf[["time", "cvd_open", "cvd_high", "cvd_low", "cvd_close"]]
cvd_df = cvd_df.rename(columns={'cvd_open': 'open', 'cvd_high': 'high', 'cvd_low': 'low', 'cvd_close': 'close'})

chart = JupyterChart(width=1000, height=300)
chart.legend(visible=True)
chart.set(cvd_df)

chart.load()

In [ ]:
candles_df